# Floating point in the news, or becoming an amateur numerical detective by programming in Julia

April 19, 2020

#### In this notebook

I will talk about a news article that nerdsniped me into using Julia, a really performant and easy programming language, to talk about the very first steps into the field of numerical analysis, aka, floating point arithmetic, and how Julia is a good fit for reasoning about these types of programs. You should be able to press `Shift+Enter` to run the code yourself, and add new cells with the toolbar. Try it out, and happy hacking!

-mrg

#### Things you'll learn

- How to know get basic information about the numbers you use in Julia
- Brief discussion about different ways of representing numbers with binary digits (bits!)

#### Expected time: ~ 10 minutes

#### Previous experience: None! All are welcome!

You might have heard of a recent incident of planes needing to be turned on and off in order to combat a [nasty software bug](https://www.theregister.co.uk/2020/04/02/boeing_787_power_cycle_51_days_stale_data/):



![pic](pic.png)

Yikes. That's not cool. People on the friendly Julia (which you can join by [clicking here](https://slackinvite.julialang.org/), and ask questions to your heart's delight) were in a collective groan when I believe @dextorious suggested this was a software bug that had to with keeping the timers in milliseconds.

### lolwut

This should sound bananas - Why would keeping the timer in milliseconds mean that a plane should be "turned on and off again"?

To take a stab at answering that question, we need to understand they way in which computers deal with numbers, or at least the bare minimum introduction. This field includes Numerical Analysis and if you are interested, there is a free online course from MIT that [you can follow here](https://github.com/mitmath/18330). 

### Into the DevilIEEEsh details

Computers have finite memory, and thus in order to store numbers, and have some notion of them being "continuous", something will have to give. There are many ways to do this, but it is devilishly hard to come up with a scheme that doesn't run into hellraising contradictions, see the [recent XKCD](https://xkcd.com/2295/) comic for an inkling into how this goes.

This means that there is some limit into how many numbers you can store in a certain amount of bits. If your computer happens to be a 64 bit architecture, you can store up to `2^64 - 1` different numbers.

In [53]:
big(2)^64 - 1

18446744073709551615

I used the `big(2)` so that Julia uses as many digits as it needs to perform that computation. Because Julia is an interactive programming language, I don't need to compile everytime I want to know something, I just need to ask Julia with a `?`:

In [22]:
?maxintfloat(Float64)

```
maxintfloat(T=Float64)
```

The largest consecutive integer-valued floating-point number that is exactly represented in the given floating-point type `T` (which defaults to `Float64`).

That is, `maxintfloat` returns the smallest positive integer-valued floating-point number `n` such that `n+1` is *not* exactly representable in the type `T`.

When an `Integer`-type value is needed, use `Integer(maxintfloat(T))`.

---

```
maxintfloat(T, S)
```

The largest consecutive integer representable in the given floating-point type `T` that also does not exceed the maximum integer representable by the integer type `S`.  Equivalently, it is the minimum of `maxintfloat(T)` and [`typemax(S)`](@ref).


In [23]:
# I could also try the typemax function:
typemax(Int32)

2147483647

And we can write a neat for loop for writing out the info on many of these "out of the box types":

In [64]:
types = (Int8, UInt8, Int16, UInt16, Int32, UInt32, Int64, UInt64)
for t in types
    println(t, " ",typemax(t))
end

Int8 127
UInt8 255
Int16 32767
UInt16 65535
Int32 2147483647
UInt32 4294967295
Int64 9223372036854775807
UInt64 18446744073709551615


### Hmmm...

Now this starts to smell a bit fishy. It turns out, many programming languages and computer architectures default to using 32 bits for all their computation, which (let's guess!) can at most represent 2,147,483,647 distinct values. This might be a clue related to the nasty software bug that started this. Again, we can ask Julia what that looks like inside the machine with the `bitstring` function:

In [24]:
bitstring(typemax(Int32))

"01111111111111111111111111111111"

#### Whoops!

Why is it that the maximum number it says it can represent doesn't use that first bit? It's missing out on a ton of digits!

There's a good reason for this: bits are very, *very* expensive real estate when it comes down to computing, and it does make sense to cram in as much information as possible - but as always, there are tradeoffs, and you want to also be able to represent negative integers in the same format. You can, however, interpret those same 32 bits as a n integer format but without a sign bit easily as well, known as an *Unsigned integer*:

In [32]:
typemax(UInt32) |> big # just some formatting shenanigans

4294967295

In [29]:
bitstring(typemax(UInt32))

"11111111111111111111111111111111"

#### Key takeaway!

The same bitstring can represent wildly different things. The first `Int32` represents Signed Integers up to 32 bits (with 1 for a sign bit), and `UInt32` means Unsigned Integer up to 32 bits. The important thing to keep in mind is that the *interpretation* of the bits matters, a lot, not just the bits per se. We call this interpretation it's data type. Tons of optimizations are available if you cram the information into the least amounts of bits possible, but you have to be **very** careful not to mess up the interpretations.

# So what's the bug?

Let's try and hunt it down! First we'll write a function to calculate the number of seconds in a day:

In [38]:
# 1000 milliseconds in a second * 60 seconds in minute * 60 minutes in 1 hour * 24  hours in n days
millis_in_days(n) = 1000*60*60*24*n
millis_in_days(1)

86400000

Now, the article said the planes needed to 51 days... 

In [50]:
t0 = millis_in_days(49)
t1 = millis_in_days(50)
t0,t1

(4233600000, 4320000000)

Which seems dangerously close to...

In [42]:
t2 = typemax(UInt32) |> big

4294967295

And it least _seems_ suggests a plausible explanation: whoever wrote the code, relied on the number of millimeters to not exceed the maximum of `UInt32`. However... when that does happen...

In [66]:
typemax(UInt32) + UInt32(1)

0x00000000

Which would be a sticky situation for a code that considers subsequent measurements of time to be increasing, and instead has "wrapped around" to 0.

#### This is common!

Remember, in finite memory, we have to use every bit to the max. There are a lot of useful applications to having this wrap-around behaviour - but you can also prevent it, at a great performance cost, with *over-flow* checks.

## Bonus!

The article mentions other instances of nasty software bugs - Can you find any other "likely" causes of it by inspecting numbers and how they are represented? Maybe! 

1. 787s power down every 248 days.
2. A350 Airbus had forced power cycles every 149 hours.


In [67]:
# Last hint: I will give you this function:
days_from_millis(n) = n / 86_400_000

days_from_millis (generic function with 1 method)

Try it out! If you get stuck, there's no shame in looking up the [Julia Manual for 1.4](https://docs.julialang.org/en/v1.5-dev/manual/integers-and-floating-point-numbers/) or asking friends on Slack/Discourse. We've barely scratched the surface of how computers represent numbers, there's a lot of cases to be solved if you want to jump into being a numerical detective!

### Extra extra bonus points

We basically used Julia here for some glorified calculations - that's cool, but not exactly revolutionary. What would be really cool would be to extend coders the ability to define their own representations of any floating point number, calculate stuff, and have it be performant. 

That's a much, much deeper dive, but if you feel like peeking into the numerical rabbithole, have a go and look at defining an number based on an [arbitrary radix, like ArbitraryRadixFloatingPoint.jl does](https://github.com/jiahao/ArbRadixFloatingPoints.jl), a package by Dr. Jiahao Chen.